In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from PIL import Image

quant_config = BitsAndBytesConfig(
    load_in_4bit=True
)

model_folder = "/mnt/resource/blobfuse2tmp/nando-mlops-vision-prod/models/office/nando-eye/experiments/minicpm-v-2_6/20240903_2/output/merged_model/"

model = AutoModelForCausalLM.from_pretrained(model_folder, quantization_config = quant_config, trust_remote_code=True, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_folder, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
model.eval()

MiniCPMV(
  (llm): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151666, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
            (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
            (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
          (

In [6]:
import time

# Start time
start_time = time.time()

question_train = 'Which object is shown to the camera? Answer the question using a single word or phrase.'

image = Image.open('9ef0376d1227b1f2_03_09_2024__16_59_25.jpg').convert('RGB')
question = 'You are an expert in recognizing objects. The objects are showned to the camera by persons. Which object is showned to the camera?. Please answer with one word and not consider hands.'
msgs = [{'role': 'user', 'content': question_train}]

answer = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True
)

# End time
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

print(answer)

Execution time: 3.0820789337158203 seconds
Nothing


In [13]:
#Two images comparison
import time

# Start time
start_time = time.time()

question_train = 'Which object is shown to the camera? Answer the question using a single word or phrase.'
question_compare = 'What can you see in the two images?'
question = 'You are an expert in recognizing objects. The objects are showned to the camera by persons. Which object is showned to the camera?. Please answer with one word and not consider hands.'

image = Image.open('2697d1c510a42b54_03_09_2024__16_27_50.jpg').convert('RGB')
image_2 = Image.open('9ef0376d1227b1f2_03_09_2024__16_59_25.jpg').convert('RGB')
msgs = [{'role': 'user', 'content': question_compare}]

answer = model.chat(
    image=[image, image_2],
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True
)

# End time
end_time = time.time()

# Calculate and print the execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

print(answer)

Execution time: 7.594323635101318 seconds
The first image shows a close-up of a person's hand holding a small, dark-colored object against their face. The object appears to be a smartphone or a similar device with a screen that is turned off or not in use.

The second image depicts a large, open field under a clear blue sky during what seems to be daytime. There are no visible buildings, trees, or other distinct features in the landscape, giving it an expansive and somewhat desolate appearance.


In [15]:
#Few show tuning

question = "object in picture" 
image1 = Image.open('2697d1c510a42b54_03_09_2024__16_27_50.jpg').convert('RGB')
answer1 = "watch"
image2 = Image.open('9ef0376d1227b1f2_03_09_2024__16_59_25.jpg').convert('RGB')
answer2 = "wire"
image_test = Image.open('c037acfee1068676_03_09_2024__17_25_51.jpg').convert('RGB')

msgs = [
    {'role': 'user', 'content': [image1, question]}, {'role': 'assistant', 'content': [answer1]},
    {'role': 'user', 'content': [image2, question]}, {'role': 'assistant', 'content': [answer2]},
    {'role': 'user', 'content': [image_test, question]}
]

answer = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)
print(answer)

plastic object
